# Step 2 Handling Missing Values

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [2]:
df=pd.read_csv("Capstone_new.csv")

In [3]:
df.head(1)

,make_model,body_type,price,vat,km,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Paint Type,Nr. of Doors,Nr. of Seats,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,Gears,age,Upholstery_type,Upholstery_color,Consumption_combined,Consumption_city,Consumption_country
0,Audi A1,Sedans,15770,VAT deductible,56013.000,66.000,Used,2.000,06/2021,Yes,NaN,Metallic,5.000,5.000,Automatic,1422.000,3.000,1220.000,front,Diesel,99.000,Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...",NaN,2016.000,Cloth,Black,3.800,4.300,3.500


In [4]:
list_miss=[]
[list_miss.append(column) for column in df.columns if any(df[column].isnull())]
list_miss

['body_type',
 'vat',
 'km',
 'hp',
 'Type',
 'Previous Owners',
 'Next Inspection',
 'Inspection new',
 'Warranty',
 'Paint Type',
 'Nr. of Doors',
 'Nr. of Seats',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive chain',
 'CO2 Emission',
 'Emission Class',
 'Comfort & Convenience',
 'Entertainment & Media',
 'Extras',
 'Safety & Security',
 'Gears',
 'age',
 'Upholstery_type',
 'Upholstery_color',
 'Consumption_combined',
 'Consumption_city',
 'Consumption_country']

**Firstly we should fill the most helpful columns!**

In [5]:
def first_overview(x):
    print("column_name",x)
    print("**************")
    print("number_of_nulls", df[x].isnull().sum())
    print("per_of nulls", "%", round(df[x].isnull().sum()*100/df.shape[2],2))
    print(df[x].value_counts(dropna=False))

### 1) Age

In [6]:
df["age"]=2019-df["age"]

In [7]:
df.groupby("Type").age.value_counts(dropna=False)

Type            age  
Demonstration   0.000     632
                1.000     130
                2.000      27
                NaN         5
                3.000       2
Employee's car  1.000     765
                0.000     127
                2.000      99
                3.000      17
                NaN         3
New             NaN      1547
                0.000     100
                1.000       3
Pre-registered  0.000    1187
                1.000     161
                2.000       9
                NaN         6
                3.000       1
Used            3.000    3653
                1.000    3463
                2.000    3138
                0.000     807
                NaN        35
Name: age, dtype: int64

In [8]:
df["age"].value_counts(dropna=False)

1.000    4522
3.000    3674
2.000    3273
0.000    2853
NaN      1597
Name: age, dtype: int64

In [9]:
df.loc[df[(df["age"]==0.0) & (df["km"]>5000)].index,"age"]=1

### I thought if the "km" is more than 5000 age couldn't be 0

**I'd say here if it is Type-Used age could be 0 or 1**

In [10]:
df['age'].fillna('-', inplace=True)

**describe() does NOT return anything for "NaN" values. If you have "Nan" at the column you are interested in, you should replace these "nan" values with something else. In our case, we have replaced the "NaN" values with "-" so that we could implement the "describe" method.**

In [11]:
df.groupby("age").km.describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,2337.000,792.502,1460.188,1.000,10.000,18.000,850.000,5000.000
1.0,4853.000,17445.066,11234.899,1.000,9000.000,16680.000,24772.000,136000.000
2.0,3272.000,41754.941,28295.748,1.000,21541.750,34752.000,54805.500,317000.000
3.0,3674.000,77442.521,39170.143,10.000,48000.000,72914.500,99950.000,291800.000
-,759.000,934.497,7416.244,0.000,5.000,10.000,10.000,89982.000


In [12]:
df['age'].replace('-', np.NaN, inplace=True)

In [13]:
df.loc[df["Type"]=="New","age"]=(df.loc[df["Type"]=="New","age"]).replace(np.NaN,0)

In [14]:
df.age.value_counts(dropna=False)

1.000    4891
0.000    4031
3.000    3674
2.000    3273
NaN        50
Name: age, dtype: int64

In [15]:
df['age'].fillna('-', inplace=True)

In [16]:
df.groupby("age").km.describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,3050.000,610.190,1320.133,0.000,10.000,10.000,150.000,5000.000
1.0,4853.000,17445.066,11234.899,1.000,9000.000,16680.000,24772.000,136000.000
2.0,3272.000,41754.941,28295.748,1.000,21541.750,34752.000,54805.500,317000.000
3.0,3674.000,77442.521,39170.143,10.000,48000.000,72914.500,99950.000,291800.000
-,46.000,15223.457,26537.973,1.000,10.000,640.500,20451.000,89982.000


In [17]:
cond1=(df["km"]<10000)
cond2=((df["km"]>=10000) &(df["km"]<30000))
cond3=((df["km"]>=30000) &(df["km"]<50000))
cond4=(df["km"]>=50000)

In [18]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('-', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('-', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('-', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('-', 3)

In [19]:
df.age.value_counts(dropna=False)

1.0    4897
0.0    4062
3.0    3679
2.0    3277
-         4
Name: age, dtype: int64

In [20]:
df.groupby("age").km.describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,3081.000,614.251,1326.434,0.000,10.000,10.000,150.000,6100.000
1.0,4859.000,17445.694,11229.330,1.000,9000.000,16692.000,24763.500,136000.000
2.0,3276.000,41748.577,28279.313,1.000,21572.500,34752.000,54755.000,317000.000
3.0,3679.000,77450.063,39145.119,10.000,48000.000,72945.000,99950.000,291800.000
-,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df["age"]=df["age"].replace("-",df["age"].mode()[0])

In [22]:
df.age.value_counts(dropna=False)

1.000    4901
0.000    4062
3.000    3679
2.000    3277
Name: age, dtype: int64

### 2) VAT

In [23]:
df.vat.value_counts(dropna=False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

In [24]:
df['vat'] = df['vat'].fillna('VAT not deductible')

In [25]:
df["vat"].value_counts(dropna=False)

VAT deductible        10980
VAT not deductible     4513
Price negotiable        426
Name: vat, dtype: int64

 ### 3) Type

In [26]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64

In [27]:
df['Type'] = df['Type'].fillna('Used')

### 4) Km

In [28]:
df["km"].value_counts(dropna=False)

10.000       1045
NaN          1024
1.000         367
5.000         170
50.000        148
             ... 
67469.000       1
43197.000       1
10027.000       1
35882.000       1
57.000          1
Name: km, Length: 6690, dtype: int64

In [29]:
df["km"].fillna(df.groupby(["age","Type"])["km"].transform(lambda x: x.mean()),inplace=True)  #value_counts(dropna=False).head(200)

In [30]:
df["km"].isnull().sum()

0

In [31]:
df["km"].value_counts(dropna=False)

10.000       1045
16.987        848
1.000         367
5.000         170
50.000        148
             ... 
53433.000       1
67469.000       1
43197.000       1
10027.000       1
57.000          1
Name: km, Length: 6699, dtype: int64

In [32]:
#df.km=(round((df["km"]/1000),0)*1000)

### 5) Body Type

In [33]:
df.body_type.replace("Other",np.NaN,inplace=True)

In [34]:
df.body_type.value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

**We do NOT have any information about "other". So we have assigned them to "NaN" to use "fillna" function**

In [35]:
df.columns

Index(['make_model', 'body_type', 'price', 'vat', 'km', 'hp', 'Type',
       'Previous Owners', 'Next Inspection', 'Inspection new', 'Warranty',
       'Paint Type', 'Nr. of Doors', 'Nr. of Seats', 'Gearing Type',
       'Displacement', 'Cylinders', 'Weight', 'Drive chain', 'Fuel',
       'CO2 Emission', 'Emission Class', 'Comfort & Convenience',
       'Entertainment & Media', 'Extras', 'Safety & Security', 'Gears', 'age',
       'Upholstery_type', 'Upholstery_color', 'Consumption_combined',
       'Consumption_city', 'Consumption_country'],
      dtype='object')

In [36]:
df["body_type"]=df["body_type"].fillna(df.groupby("make_model")["body_type"].transform(lambda x:x.mode()[0]))

In [37]:
df.body_type.value_counts(dropna=False)

Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

### 6) Previous Owners

In [38]:
df.groupby(["age","km","Type"])["Previous Owners"].value_counts(dropna=False).head(500)

age    km        Type            Previous Owners
0.000  0.000     New             0.000               16
                                 NaN                  3
       1.000     Employee's car  NaN                  2
                 New             NaN                142
                                 0.000               13
                 Pre-registered  1.000               83
                                 NaN                 76
                 Used            NaN                 13
                                 1.000               11
       2.000     New             NaN                  2
                 Pre-registered  NaN                  2
                                 1.000                2
       3.000     Demonstration   NaN                  3
                                 1.000                1
                 New             NaN                  4
                 Pre-registered  1.000               24
                                 NaN                  1

**I think if the km is under 5 the car may be new car. It could be spended 5 km by test drive**

In [39]:
df.loc[df[(df["Type"]=="New") & (df["km"]<15)].index,"Previous Owners"]=0

In [40]:
df["Previous Owners"].value_counts(dropna=False)

1.000    8100
NaN      6242
0.000     792
2.000     766
3.000      17
4.000       2
Name: Previous Owners, dtype: int64

In [41]:
df.loc[df[df["km"]<15].index,"Previous Owners"]=0.0

In [42]:
df["Previous Owners"].value_counts(dropna=False)

1.000    7466
NaN      5711
0.000    1957
2.000     766
3.000      17
4.000       2
Name: Previous Owners, dtype: int64

In [43]:
df.loc[((df["km"]>=15) & (df["km"]<150000) & df["Previous Owners"].isnull()),"Previous Owners"]=1.0

In [44]:
df["Previous Owners"].value_counts(dropna=False)

1.000    13137
0.000     1957
2.000      766
NaN         40
3.000       17
4.000        2
Name: Previous Owners, dtype: int64

In [45]:
df["Previous Owners"].fillna(df.groupby(['age'])['Previous Owners'].transform(lambda x: x.mode()[0]), inplace=True)

In [46]:
df["Previous Owners"].value_counts(dropna=False)

1.000    13177
0.000     1957
2.000      766
3.000       17
4.000        2
Name: Previous Owners, dtype: int64

### 7) Warranty

In [47]:
df["Warranty"].fillna("-", inplace = True)

In [48]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

count      mean      std       min       25%  \
make_model     age   Warranty                                                  
Audi A1        0.000 12.0      33.000 23968.515 2803.633 20881.000 21000.000   
                     18.0       1.000 19995.000      NaN 19995.000 19995.000   
                     24.0      99.000 23860.667 3347.693 14900.000 21890.000   
                     36.0      22.000 25204.818 3186.670 19900.000 22339.250   
                     48.0      17.000 24708.824 2316.263 19650.000 22800.000   
                     56.0       1.000 21760.000      NaN 21760.000 21760.000   
                     60.0      11.000 21695.455  557.519 20990.000 21390.000   
                     -        541.000 23833.449 3422.677 15980.000 21406.000   
               1.000 6.0        9.000 22166.667  743.303 20900.000 22400.000   
                     12.0      69.000 18463.913 2847.804 14220.000 16499.000   
                     13.0       1.000 19840.000      NaN 19840.000 19840.000   
                     20.0       1.000 17950.000      NaN 17950.000 17950.000   
                     23.0       1.000 20950.000      NaN 20950.000 20950.000   
                     24.0      96.000 18730.135 2895.494 14500.000 16487.500   
                     26.0       1.000 15840.000      NaN 15840.000 15840.000   
                     36.0      30.000 19239.100 3972.387 15500.000 16282.500   
                     46.0       1.000 15980.000      NaN 15980.000 15980.000   
                     47.0       1.000 16370.000      NaN 16370.000 16370.000   
                     48.0      27.000 17977.037 2064.101 14900.000 15900.000   
                     49.0       1.000 16270.000      NaN 16270.000 16270.000   
                     50.0       3.000 17903.333 1342.882 16370.000 17420.000   
                     60.0      14.000 17689.571 2579.654 15900.000 16047.500   
                     -        573.000 19118.972 3250.984 13450.000 16470.000   
               2.000 3.0        2.000 20740.000 3889.087 17990.000 19365.000   
                     6.0        3.000 18443.333 2475.284 15990.000 17195.000   
                     12.0      80.000 16187.125 1918.723 13500.000 14950.000   
                     24.0      33.000 16858.455 2723.500 11990.000 14990.000   
                     36.0       8.000 17129.250 2677.735 14400.000 14880.000   
                     48.0       4.000 19800.000  115.470 19700.000 19700.000   
                     60.0       1.000 14990.000      NaN 14990.000 14990.000   
                     -        301.000 16621.209 2131.491 10999.000 15280.000   
               3.000 3.0        2.000 13235.000  346.482 12990.000 13112.500   
                     6.0        9.000 15632.000 2329.396 12980.000 13990.000   
                     12.0     120.000 14324.950 1914.102  9950.000 12990.000   
                     23.0       1.000 12170.000      NaN 12170.000 12170.000   
                     24.0      13.000 14540.000 1970.787 12500.000 13000.000   
                     60.0       4.000 13467.250  391.157 12999.000 13217.250   
                     -        480.000 14526.779 1877.216  8999.000 13400.000   
Audi A2        1.000 -          1.000 28200.000      NaN 28200.000 28200.000   
Audi A3        0.000 11.0       1.000 26795.000      NaN 26795.000 26795.000   
                     12.0      39.000 24226.359 1468.979 21900.000 22900.000   
                     19.0       2.000 26273.000  388.909 25998.000 26135.500   
                     24.0      85.000 25308.365 4677.817 20290.000 23900.000   
                     36.0      70.000 25636.414 1574.653 20000.000 24530.000   
                     48.0      19.000 24593.684 1326.973 21900.000 24100.000   
                     60.0       4.000 23942.500 2531.737 20990.000 22490.000   
                     72.0       1.000 23990.000      NaN 23990.000 23990.000   
                     -        540.000 24858.443 2975.695 18820.000 23500.000   
               1.000 3.0        

**There are** **``too many nan values``** **and when we analyzed these nan values according to the price, age and make_model columns, we decided that this column does not have healthy data.**

In [49]:
df.drop("Warranty", axis=1, inplace=True)

### 8) Vat

In [50]:
df["vat"].fillna("-", inplace = True)

In [51]:
df.groupby(["make_model", "body_type", "vat"]).price.describe()

count      mean       std  \
make_model     body_type     vat                                               
Audi A1        Compact       Price negotiable      3.000 17631.667  1548.954   
                             VAT deductible      779.000 20018.999  4603.192   
                             VAT not deductible  257.000 16692.856  3377.345   
               Coupe         VAT deductible        2.000 14925.000  1378.858   
               Sedans        Price negotiable     78.000 16224.308  3545.634   
                             VAT deductible      975.000 19370.279  4470.516   
                             VAT not deductible  498.000 17650.865  4225.473   
               Station wagon VAT deductible       17.000 16747.706  2481.100   
                             VAT not deductible    4.000 22332.000  7763.736   
               Van           VAT deductible        1.000 29000.000       NaN   
Audi A2        Off-Road      VAT deductible        1.000 28200.000       NaN   
Audi A3        Compact       Price negotiable      3.000 22613.333  8141.992   
                             VAT deductible      125.000 18900.496  7543.020   
                             VAT not deductible   54.000 19112.074  4680.287   
               Convertible   Price negotiable      1.000 20000.000       NaN   
                             VAT deductible        4.000 29698.250 19403.979   
                             VAT not deductible    3.000 28150.000  5598.437   
               Coupe         Price negotiable      1.000 18000.000       NaN   
                             VAT deductible        1.000 19000.000       NaN   
                             VAT not deductible    2.000 16990.000     0.000   
               Sedans        Price negotiable     77.000 20966.844  7574.507   
                             VAT deductible     1858.000 21368.267  4137.415   
                             VAT not deductible  686.000 20583.682  5993.643   
               Station wagon Price negotiable     19.000 19245.947  2506.636   
                             VAT deductible      221.000 21130.882  4930.360   
                             VAT not deductible   42.000 18985.476  3462.126   
Opel Astra     Compact       Price negotiable      1.000 15950.000       NaN   
                             VAT deductible       68.000 19521.515  8593.552   
                             VAT not deductible  116.000 14254.681  2321.570   
               Coupe         VAT deductible        1.000 29990.000       NaN   
                             VAT not deductible    1.000 14490.000       NaN   
               Off-Road      VAT deductible        1.000 18590.000       NaN   
               Sedans        Price negotiable     23.000 11777.826  2423.031   
                             VAT deductible      758.000 16788.747  4564.558   
                             VAT not deductible  272.000 14016.423  3038.945   
               Station wagon Price negotiable     21.000 13363.762  3113.582   
                             VAT deductible     1000.000 16088.194  5463.598   
                             VAT not deductible  264.000 14243.466  3743.119   
Opel Corsa     Compact       Price negotiable      3.000  9496.333  3943.048   
                             VAT deductible     1056.000 12241.564  2992.755   
                             VAT not deductible  260.000  9307.596  2059.783   
               Coupe         Price negotiable      1.000  7500.000       NaN   
                             VAT deductible        2.000  9075.000  2651.650   
                             VAT not deductible   10.000  8839.000  1529.012   
               Off-Road      VAT not deductible    3.000  8166.667   461.880   
               Sedans        Price negotiable     37.000  9390.189  2119.206   
                             VAT deductible      544.000 10704.445  2823.441   
                             VAT not deductible  294.000  9385.728  2037.128   
               Transporter   VAT deductible        

**It is not appear to be a significant relationship between vat and price column. We can drop this column or fill missing values with ffill/bfill method to maintain the current proportionality.**

**I prefer to drop this column**

**Be informed that you can create different approaches to deal with missing values at** **``"vat"``** **column.**

In [52]:
df.drop("vat", axis=1, inplace=True)

### 9) Type

In [53]:
df['Type'] = df['Type'].fillna('Used')

### 10) Next Inspection 

In [54]:
df.drop(["Next Inspection"],axis=1,inplace=True)

### 11) Inspection New

In [55]:
df['Inspection new'] = df['Inspection new'].fillna('No')

### 12)  Paint Type

**According to the domain knowledge we know that; The colors do not affect the prices of cars directly, but their "paint types", such as whether they are metallic or not, can affect the price.**

In [56]:
df["Paint Type"].fillna("-", inplace = True)

In [57]:
df.groupby(["make_model", "body_type", "age", 'Paint Type']).price.describe()

count      mean       std  \
make_model     body_type     age   Paint Type                                
Audi A1        Compact       0.000 -           137.000 22007.730  3249.081   
                                   Metallic    200.000 24680.875  3154.727   
                                   Uni/basic     2.000 19888.000     0.000   
                             1.000 -            64.000 18312.562  3335.138   
                                   Metallic    241.000 19506.191  3258.856   
                             2.000 -            52.000 15750.115  1218.358   
                                   Metallic    108.000 17001.352  2294.777   
                                   Uni/basic     1.000 17900.000       NaN   
                             3.000 -            58.000 14007.948  1921.949   
                                   Metallic    174.000 14723.218  1881.559   
                                   Uni/basic     2.000 13200.000   424.264   
               Coupe         2.000 Metallic      1.000 15900.000       NaN   
                             3.000 Metallic      1.000 13950.000       NaN   
               Sedans        0.000 -           122.000 23072.352  3405.069   
                                   Metallic    247.000 24509.425  3014.303   
                                   Perl effect   1.000 28290.000       NaN   
                                   Uni/basic    13.000 25167.692  3043.759   
                             1.000 -           118.000 19626.331  3525.819   
                                   Metallic    369.000 18481.602  2848.881   
                                   Uni/basic    25.000 20550.280  2994.032   
                             2.000 -           101.000 16495.723  2442.793   
                                   Metallic    150.000 16682.987  2198.296   
                                   Uni/basic    16.000 17000.562  1564.755   
                             3.000 -           152.000 14003.164  1888.368   
                                   Metallic    234.000 14757.748  1832.507   
                                   Uni/basic     3.000 15276.667  2349.901   
               Station wagon 0.000 -             1.000 28999.000       NaN   
                                   Metallic      1.000 28890.000       NaN   
                             1.000 Metallic     10.000 17007.000  1794.164   
                                   Uni/basic     1.000 21450.000       NaN   
                             2.000 Metallic      3.000 17613.333  2953.817   
                             3.000 -             3.000 13642.667   599.950   
                                   Metallic      2.000 15431.000  1401.486   
               Van           0.000 Metallic      1.000 29000.000       NaN   
Audi A2        Off-Road      1.000 Metallic      1.000 28200.000       NaN   
Audi A3        Compact       0.000 -             6.000 24436.667  2127.755   
                                   Metallic     10.000 35128.300  6361.962   
                             1.000 -            19.000 21381.000  2241.854   
                                   Metallic     15.000 22397.200  5776.048   
                             2.000 -            14.000 18611.143  2186.171   
                                   Metallic     24.000 21606.458 10254.727   
                                   Uni/basic     1.000 19990.000       NaN   
                             3.000 -            34.000 15172.559  1708.284   
                                   Metallic     57.000 15368.175  2519.282   
                                   Uni/basic     2.000 15700.000  3959.798   
               Convertible   1.000 -             1.000 10893.000       NaN   
                                   Metallic      2.000 39050.000 24112.341   
                             2.000 -             2.000 31225.000  2439.518   
                                   Metallic      2.000 25900.000  7071.068   
                             3.000 -             1.000 20000.000       

In [58]:
df.groupby(["make_model", "body_type", "age"])['Paint Type'].transform(lambda x: x.mode()[0])

0        Metallic
1        Metallic
2        Metallic
3        Metallic
4        Metallic
           ...   
15914    Metallic
15915    Metallic
15916    Metallic
15917    Metallic
15918    Metallic
Name: Paint Type, Length: 15919, dtype: object

**It is not appear to be a significant relationship between Paint_Type and price column. We can drop this column or fill missing values with ffill/bfill method to maintain the current proportionality.**

**I prefer to fill missing values** 

In [59]:
df["Paint Type"].replace("-",np.NaN,inplace=True)

In [60]:
for group1 in df["make_model"].unique():
    for group2 in list(df["body_type"].unique()):
        cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
        df.loc[cond2, "Paint Type"] = df.loc[cond2, "Paint Type"].fillna(method="ffill").fillna(method="bfill")
                
for group1 in list(df["make_model"].unique()):
    cond1 = df["make_model"]==group1
    df.loc[cond1, "Paint Type"] = df.loc[cond1, "Paint Type"].fillna(method="ffill").fillna(method="bfill")            
           
df["Paint Type"] = df["Paint Type"].fillna(method="ffill").fillna(method="bfill")

In [61]:
df["Paint Type"].value_counts(dropna=False)

Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint Type, dtype: int64

### 13) Upholstery_type

In [62]:
df["Upholstery_type"].replace(np.NaN,"-",inplace=True)

In [63]:
df.groupby(["make_model", "body_type", "age", "Upholstery_type"]).price.describe()

count      mean  \
make_model     body_type     age   Upholstery_type                     
Audi A1        Compact       0.000 -                36.000 21087.278   
                                   Cloth           288.000 23909.646   
                                   Part leather      3.000 28826.667   
                                   Velour           12.000 21617.500   
                             1.000 -                60.000 18654.833   
                                   Cloth           214.000 18917.051   
                                   Full leather      4.000 21500.000   
                                   Part leather     24.000 23279.083   
                                   Velour            2.000 15899.500   
                                   alcantara         1.000 28960.000   
                             2.000 -                34.000 15516.471   
                                   Cloth           118.000 16737.017   
                                   Part leather      5.000 19091.000   
                                   Velour            1.000 15249.000   
                                   alcantara         3.000 19940.000   
                             3.000 -                26.000 13831.577   
                                   Cloth           187.000 14502.155   
                                   Full leather      2.000 17365.000   
                                   Part leather     16.000 15287.062   
                                   alcantara         3.000 16618.000   
               Coupe         2.000 Part leather      1.000 15900.000   
                             3.000 -                 1.000 13950.000   
               Sedans        0.000 -               154.000 23898.636   
                                   Cloth           218.000 23982.821   
                                   Full leather      3.000 27966.667   
                                   Part leather      6.000 30383.333   
                                   Velour            2.000 24640.000   
                             1.000 -               164.000 18763.878   
                                   Cloth           321.000 18639.458   
                                   Full leather      6.000 18386.500   
                                   Part leather     16.000 23999.000   
                                   Velour            2.000 18275.000   
                                   alcantara         3.000 19326.667   
                             2.000 -               134.000 16677.515   
                                   Cloth           115.000 16367.235   
                                   Full leather      6.000 17628.333   
                                   Part leather      5.000 18731.400   
                                   Velour            2.000 15585.000   
                                   alcantara         5.000 18581.800   
                             3.000 -               120.000 14480.767   
                                   Cloth           220.000 14267.218   
                                   Full leather      3.000 16760.000   
                                   Part leather     40.000 15272.875   
                                   Velour            4.000 13542.250   
                                   alcantara         2.000 17890.000   
               Station wagon 0.000 -                 1.000 28999.000   
                                   Part leather      1.000 28890.000   
                             1.000 -                 2.000 18070.000   
                                   Cloth             9.000 17264.444   
                             2.000 Cloth             3.000 17613.333   
                             3.000 -                 1.000 13999.000   
                                   Cloth             3.000 13789.667   
                                   Part leather      1.000 16422.000   
               Van           0.000 -                 1.000 29000.000   
Audi A2        Off-Ro

**I have decided to combine upholstery_types to reduce the number of categories.**

In [64]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], 
                              ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)

In [65]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth                8483
-                    4871
Part/Full Leather    2565
Name: Upholstery_type, dtype: int64

In [66]:
df.groupby(["make_model", "body_type", "age", "Upholstery_type"]).price.describe()

count      mean  \
make_model     body_type     age   Upholstery_type                       
Audi A1        Compact       0.000 -                  36.000 21087.278   
                                   Cloth             300.000 23817.960   
                                   Part/Full Leather   3.000 28826.667   
                             1.000 -                  60.000 18654.833   
                                   Cloth             216.000 18889.111   
                                   Part/Full Leather  29.000 23229.586   
                             2.000 -                  34.000 15516.471   
                                   Cloth             119.000 16724.513   
                                   Part/Full Leather   8.000 19409.375   
                             3.000 -                  26.000 13831.577   
                                   Cloth             187.000 14502.155   
                                   Part/Full Leather  21.000 15675.095   
               Coupe         2.000 Part/Full Leather   1.000 15900.000   
                             3.000 -                   1.000 13950.000   
               Sedans        0.000 -                 154.000 23898.636   
                                   Cloth             220.000 23988.795   
                                   Part/Full Leather   9.000 29577.778   
                             1.000 -                 164.000 18763.878   
                                   Cloth             323.000 18637.201   
                                   Part/Full Leather  25.000 22091.320   
                             2.000 -                 134.000 16677.515   
                                   Cloth             117.000 16353.863   
                                   Part/Full Leather  16.000 18271.000   
                             3.000 -                 120.000 14480.767   
                                   Cloth             224.000 14254.272   
                                   Part/Full Leather  45.000 15488.333   
               Station wagon 0.000 -                   1.000 28999.000   
                                   Part/Full Leather   1.000 28890.000   
                             1.000 -                   2.000 18070.000   
                                   Cloth               9.000 17264.444   
                             2.000 Cloth               3.000 17613.333   
                             3.000 -                   1.000 13999.000   
                                   Cloth               3.000 13789.667   
                                   Part/Full Leather   1.000 16422.000   
               Van           0.000 -                   1.000 29000.000   
Audi A2        Off-Road      1.000 Cloth               1.000 28200.000   
Audi A3        Compact       0.000 -                   4.000 25582.500   
                                   Cloth               4.000 22863.750   
                                   Part/Full Leather   8.000 38014.750   
                             1.000 -                  15.000 21976.600   
                                   Cloth               6.000 20814.833   
                                   Part/Full Leather  13.000 22127.615   
                             2.000 -                  19.000 19352.421   
                                   Cloth              12.000 18463.750   
                                   Part/Full Leather   8.000 26230.000   
                             3.000 -                  66.000 14683.848   
                                   Cloth              22.000 16115.409   
                                   Part/Full Leather   5.000 19916.000   
               Convertible   1.000 -                   1.000 22000.000   
                                   Part/Full Leather   2.000 33496.500   
                             2.000 -                   1.000 29500.000   
                                   Cloth               2.000 25900.000   
                                   Part/Full Leather   1.00

**I haven't detected significant relationship to fill missing values.**

**I prefer to fill missing values with ffill/bfill method to maintain the current proportionality.**

In [67]:
df["Upholstery_type"].replace("-", np.nan, inplace=True)

In [68]:
def fill(df, group_col1, group_col2, col_name, method): # method can be either "mode" or "mean" or "median" or "ffill"
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [69]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

Number of NaN :  0
------------------
Cloth                12238
Part/Full Leather     3681
Name: Upholstery_type, dtype: int64


In [70]:
df.drop("Upholstery_color", axis=1, inplace=True)

### 14) Number of Door

In [71]:
def fill_mode(df, group_col1, group_col2, col_name):
    
    for group1 in list(df[group_col1].unique()):
        for group2 in list(df[group_col2].unique()):
            cond1 = df[group_col1]==group1
            cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
            mode1 = list(df[cond1][col_name].mode())
            mode2 = list(df[cond2][col_name].mode())
            if mode2 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
            elif mode1 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
            else:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
    
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [72]:
fill_mode(df, "make_model", "body_type", "Nr. of Doors")

Number of NaN :  0
------------------
5.000    11787
4.000     3079
3.000      832
2.000      219
1.000        1
7.000        1
Name: Nr. of Doors, dtype: int64


### 15) Number of Seats

In [73]:
df.columns

Index(['make_model', 'body_type', 'price', 'km', 'hp', 'Type',
       'Previous Owners', 'Inspection new', 'Paint Type', 'Nr. of Doors',
       'Nr. of Seats', 'Gearing Type', 'Displacement', 'Cylinders', 'Weight',
       'Drive chain', 'Fuel', 'CO2 Emission', 'Emission Class',
       'Comfort & Convenience', 'Entertainment & Media', 'Extras',
       'Safety & Security', 'Gears', 'age', 'Upholstery_type',
       'Consumption_combined', 'Consumption_city', 'Consumption_country'],
      dtype='object')

**The number of seats of cars changes by theirs make_models and body_types. So I have decided to fill missing values with mode value of related group.**

In [74]:
fill(df, "make_model", "body_type", "Nr. of Seats", "mode")

Number of NaN :  0
------------------
5.000    14308
4.000     1127
7.000      362
2.000      119
6.000        2
3.000        1
Name: Nr. of Seats, dtype: int64


### 16 )Cylinders

In [75]:
df["Cylinders"].fillna("-", inplace=True)

In [76]:
df.groupby(["make_model", "body_type", "age", "Cylinders"]).price.describe()

count      mean       std  \
make_model     body_type     age   Cylinders                               
Audi A1        Compact       0.000 3.0       164.000 23833.323  3529.433   
                                   4.0        18.000 25435.056  3358.390   
                                   -         157.000 23086.076  3292.892   
                             1.000 3.0       152.000 19277.454  3230.954   
                                   4.0        35.000 21460.800  2456.567   
                                   -         118.000 18573.686  3352.531   
                             2.000 3.0        60.000 16555.550  1858.121   
                                   4.0        31.000 17756.548  2534.626   
                                   -          70.000 16132.371  1874.650   
                             3.000 3.0        75.000 14363.520  1942.498   
                                   4.0        26.000 15136.423  1971.253   
                                   8.0         1.000 12990.000       NaN   
                                   -         132.000 14521.970  1871.131   
               Coupe         2.000 -           1.000 15900.000       NaN   
                             3.000 3.0         1.000 13950.000       NaN   
               Sedans        0.000 3.0       286.000 24099.098  3186.584   
                                   4.0        10.000 23650.100  4345.332   
                                   -          87.000 24083.701  3209.585   
                             1.000 3.0       274.000 18502.730  3279.445   
                                   4.0       119.000 19405.849  2713.855   
                                   -         119.000 19078.412  2865.162   
                             2.000 3.0       151.000 16245.358  2043.877   
                                   4.0        67.000 17322.478  2415.989   
                                   -          49.000 16874.898  2452.331   
                             3.000 3.0       185.000 13974.984  1796.640   
                                   4.0        84.000 15577.690  2173.294   
                                   -         120.000 14447.717  1455.113   
               Station wagon 0.000 -           2.000 28944.500    77.075   
                             1.000 3.0         8.000 17210.000  1968.930   
                                   4.0         2.000 18975.000  3500.179   
                                   -           1.000 15890.000       NaN   
                             2.000 3.0         1.000 14750.000       NaN   
                                   4.0         2.000 19045.000  2269.813   
                             3.000 3.0         1.000 14440.000       NaN   
                                   4.0         2.000 15200.500  1727.462   
                                   -           2.000 13474.500   741.755   
               Van           0.000 -           1.000 29000.000       NaN   
Audi A2        Off-Road      1.000 -           1.000 28200.000       NaN   
Audi A3        Compact       0.000 3.0         9.000 36310.333  5460.344   
                                   4.0         7.000 24444.286  1942.470   
                             1.000 3.0         3.000 26949.667 11275.597   
                                   4.0        28.000 21255.321  2750.281   
                                   -           3.000 22066.333  3134.135   
                             2.000 3.0         5.000 18625.000   939.162   
                                   4.0        21.000 19836.190  3309.448   
                                   5.0         1.000 67600.000       NaN   
                                   -          12.000 18484.667  1996.178   
                             3.000 4.0        80.000 15131.688  2280.331   
                                   -          13.000 16362.923  1843.732   
               Convertible   1.000 4.0         3.000 29664.333 23557.901   
                             2.000 4.0         4.000 28562.500  5301.159   
                   

**There are other features having less missing values and giving similar insight such as "hp, weight, displacement". So I have decided to drop this column.**

In [77]:
df.drop("Cylinders", axis=1, inplace=True)

### 17) Drive chain

In [78]:
df["Drive chain"].fillna("-", inplace=True)

In [79]:
df.groupby(["make_model", "body_type", "Drive chain"]).price.describe()

count      mean       std  \
make_model     body_type     Drive chain                                
Audi A1        Compact       -            352.000 17620.869  4226.116   
                             4WD            2.000 14790.000  1258.650   
                             front        685.000 20008.223  4511.348   
               Coupe         -              2.000 14925.000  1378.858   
               Sedans        -            561.000 17830.440  4362.321   
                             4WD            1.000 15450.000       NaN   
                             front        989.000 19133.794  4441.969   
               Station wagon -              3.000 24593.000  7537.216   
                             front         18.000 16681.111  2493.673   
               Van           front          1.000 29000.000       NaN   
Audi A2        Off-Road      front          1.000 28200.000       NaN   
Audi A3        Compact       -             25.000 18236.520  3427.236   
                             4WD            1.000 67600.000       NaN   
                             front        156.000 18839.365  6078.743   
               Convertible   -              4.000 26112.500  6123.775   
                             4WD            1.000 56100.000       NaN   
                             front          3.000 20897.667 10003.500   
               Coupe         -              1.000 16990.000       NaN   
                             front          3.000 17996.667  1005.004   
               Sedans        -            891.000 20787.714  5846.561   
                             4WD           17.000 32913.765 20353.178   
                             front       1713.000 21223.413  3546.978   
               Station wagon -            149.000 20254.745  5473.077   
                             front        133.000 21165.647  3552.001   
Opel Astra     Compact       -             81.000 17993.346  6740.198   
                             front        104.000 14802.837  5080.910   
               Coupe         -              2.000 22240.000 10960.155   
               Off-Road      front          1.000 18590.000       NaN   
               Sedans        -            507.000 15488.444  4123.966   
                             4WD            2.000 13075.000  3641.600   
                             front        544.000 16416.243  4600.632   
               Station wagon -            588.000 14923.395  4925.886   
                             front        697.000 16290.032  5318.146   
Opel Corsa     Compact       -            678.000 10877.917  2906.949   
                             front        641.000 12481.011  3015.968   
               Coupe         -              6.000  9198.333  1848.463   
                             front          7.000  8407.143  1342.395   
               Off-Road      -              3.000  8166.667   461.880   
               Sedans        -            359.000  9755.081  2483.224   
                             front        516.000 10519.351  2692.189   
               Transporter   -              1.000 10500.000       NaN   
                             front          6.000  7175.000  2073.343   
               Van           -              2.000  7900.000     0.000   
Opel Insignia  Compact       -             17.000 28728.353 10844.301   
                             front         10.000 28355.000 12606.283   
               Coupe         -              1.000 17094.000       NaN   
               Off-Road      -              1.000 33490.000       NaN   
                             4WD            5.000 28232.800  8948.569   
                             front          1.000 31505.000       NaN   
               Sedans        -            368.000 19794.959  6394.251   
                             4WD           26.000 34139.346  6543.818   
                             front        505.000 21675.178  7462.771   
                             rear           1.000 18990.000       NaN   
               Station wagon 

In [80]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [81]:
df.loc[cond, 'Drive chain'] = df.loc[cond, 'Drive chain'].replace('-', '4WD')

In [82]:
df["Drive chain"].value_counts(dropna=False)

front    8886
-        6826
4WD       203
rear        4
Name: Drive chain, dtype: int64

In [83]:
df["Drive chain"] = df["Drive chain"].replace('-', np.nan)

In [84]:
df["Drive chain"].value_counts(dropna=False)

front    8886
NaN      6826
4WD       203
rear        4
Name: Drive chain, dtype: int64

In [85]:
fill(df, "make_model", "body_type", "Drive chain", "mode")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: Drive chain, dtype: int64


### 18) Emission Class

In [86]:
df.drop("Emission Class", axis=1, inplace=True)

In [87]:
df.columns

Index(['make_model', 'body_type', 'price', 'km', 'hp', 'Type',
       'Previous Owners', 'Inspection new', 'Paint Type', 'Nr. of Doors',
       'Nr. of Seats', 'Gearing Type', 'Displacement', 'Weight', 'Drive chain',
       'Fuel', 'CO2 Emission', 'Comfort & Convenience',
       'Entertainment & Media', 'Extras', 'Safety & Security', 'Gears', 'age',
       'Upholstery_type', 'Consumption_combined', 'Consumption_city',
       'Consumption_country'],
      dtype='object')

### 19) Gears

In [88]:
df["Gears"].fillna("-", inplace=True)

In [89]:
df.groupby(["make_model", "body_type", "Gearing Type", "Gears"]).price.describe()

count      mean       std  \
make_model     body_type     Gearing Type   Gears                               
Audi A1        Compact       Automatic      5.0     3.000 22184.333  3421.302   
                                            6.0     6.000 21038.333  4282.039   
                                            7.0   199.000 22059.251  3918.116   
                                            8.0     1.000 16880.000       NaN   
                                            -     253.000 21640.427  4965.500   
                             Manual         5.0   277.000 16329.469  3040.933   
                                            6.0    77.000 20538.299  2061.004   
                                            -     220.000 16756.727  3182.991   
                             Semi-automatic 7.0     3.000 24028.333  7208.440   
               Coupe         Manual         5.0     1.000 13950.000       NaN   
                                            -       1.000 15900.000       NaN   
               Sedans        Automatic      5.0    14.000 18078.571  1329.649   
                                            6.0    11.000 22974.364  5983.728   
                                            7.0   398.000 21320.038  4146.217   
                                            8.0     1.000 20500.000       NaN   
                                            -     257.000 21100.016  4361.498   
                             Manual         5.0   513.000 15490.267  2529.015   
                                            6.0    89.000 21144.978  2273.883   
                                            7.0     1.000 15500.000       NaN   
                                            -     213.000 16629.319  2916.426   
                             Semi-automatic 6.0     1.000 37900.000       NaN   
                                            7.0    52.000 20432.635  5370.504   
                                            -       1.000 20900.000       NaN   
               Station wagon Automatic      7.0     3.000 23193.000  5069.983   
                                            -       5.000 20970.400  4904.192   
                             Manual         5.0    10.000 15574.000  1198.621   
                                            6.0     1.000 13979.000       NaN   
                                            -       2.000 14944.500  1337.139   
               Van           Automatic      6.0     1.000 29000.000       NaN   
Audi A2        Off-Road      Manual         -       1.000 28200.000       NaN   
Audi A3        Compact       Automatic      6.0     4.000 17820.000  1839.438   
                                            7.0    56.000 19340.982  3092.938   
                                            -      10.000 19448.400  2145.488   
                             Manual         5.0     1.000 18900.000       NaN   
                                            6.0    87.000 16091.356  3171.229   
                                            7.0     1.000 15000.000       NaN   
                                            -       9.000 17373.333  3100.387   
                             Semi-automatic 6.0     1.000 24950.000       NaN   
                                            7.0    13.000 38341.308  9924.338   
               Convertible   Automatic      7.0     3.000 17931.000  6119.852   
                             Semi-automatic 6.0     2.000 25450.000  7707.464   
                                            7.0     3.000 39516.667 14464.814   
               Coupe         Automatic      7.0     4.000 17745.000   962.653   
               Sedans        Automatic      6.0    72.000 18276.889  2859.820   
                                            7.0   790.000 21967.932  3273.470   
                                            -     312.000 21566.500  3834.391   
                             Manual         5.0     1.000 12949.000       NaN   
                                            6.0   935.000 19965.568  3279.827   
  

**The number of gears of cars changes by theirs make_models, body_types and gear_types most of the time. So I have decided to fill missing values with mode value of related group.**

In [90]:
df["Gears"].replace("-", np.nan, inplace=True)

In [91]:
for group1 in df["make_model"].unique():
    for group2 in df["body_type"].unique():
        for group3 in df["Gearing Type"].unique():
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

In [92]:
df["Gears"].value_counts(dropna=False)

6.000    8615
5.000    4256
7.000    2810
8.000     225
9.000       6
1.000       2
3.000       2
4.000       2
2.000       1
Name: Gears, dtype: int64

### 20)  hp

In [93]:
df["hp"].fillna("-", inplace=True)

In [94]:
df.groupby(["make_model", "body_type","hp"]).price.describe()

count      mean       std       min  \
make_model     body_type     hp                                             
Audi A1        Compact       60.0    58.000 15189.828  1663.611 10900.000   
                             66.0   162.000 15398.210  1934.301 10490.000   
                             70.0   332.000 17983.262  3315.814  9950.000   
                             71.0    32.000 20831.594  2410.840 15890.000   
                             85.0   330.000 22604.103  4572.568 11100.000   
                             86.0     1.000 14295.000       NaN 14295.000   
                             92.0    90.000 18029.056  3027.431 12550.000   
                             93.0     2.000 21447.500   774.282 20900.000   
                             110.0   20.000 23620.450  3777.344 15490.000   
                             141.0    2.000 22495.000   841.457 21900.000   
                             147.0    8.000 28797.125   355.356 28200.000   
                             -        2.000 19125.000  5126.524 15500.000   
               Coupe         70.0     1.000 13950.000       NaN 13950.000   
                             85.0     1.000 15900.000       NaN 15900.000   
               Sedans        60.0    95.000 15218.168  1624.267 11900.000   
                             66.0   296.000 15922.797  2331.608 10800.000   
                             67.0    12.000 15432.417  2778.191 11990.000   
                             70.0   455.000 17282.068  3210.456  8999.000   
                             71.0    24.000 19177.250  2539.495 14390.000   
                             85.0   555.000 21609.523  4822.007 10000.000   
                             87.0     4.000 14865.000   750.000 14490.000   
                             92.0    79.000 19078.975  2790.209 13000.000   
                             93.0     1.000 17900.000       NaN 17900.000   
                             110.0    9.000 22591.333  4188.635 16999.000   
                             112.0    1.000 18399.000       NaN 18399.000   
                             140.0    1.000 23490.000       NaN 23490.000   
                             141.0   14.000 21232.000  2597.466 14500.000   
                             147.0    5.000 30729.600  4008.782 28880.000   
               Station wagon 60.0     3.000 15106.667  1890.882 12950.000   
                             66.0     7.000 16018.429  1779.762 13999.000   
                             70.0     3.000 17420.000  3283.474 14440.000   
                             85.0     4.000 22702.750  7207.560 16422.000   
                             92.0     3.000 17623.000  3738.860 13979.000   
                             110.0    1.000 20650.000       NaN 20650.000   
               Van           147.0    1.000 29000.000       NaN 29000.000   
Audi A2        Off-Road      85.0     1.000 28200.000       NaN 28200.000   
Audi A3        Compact       81.0    50.000 17147.920  2283.775 12900.000   
                             82.0    11.000 17042.091  1353.481 14450.000   
                             85.0    53.000 22656.717  6328.251 13790.000   
                             87.0     3.000 22195.000   349.887 21990.000   
                             92.0    43.000 13824.884   984.708 13100.000   
                             110.0   15.000 25249.333  7908.351 13990.000   
                             112.0    1.000 17894.000       NaN 17894.000   
                             135.0    3.000 15950.000     0.000 15950.000   
                             150.0    1.000 24950.000       NaN 24950.000   
                             294.0    1.000 67600.000       NaN 67600.000   
                             -        1.000 18700.000       NaN 18700.000   
               Convertible   81.0     1.000 20900.000       NaN 20900.000   
                             110.0    5.000 24848.600  9248.721 10893.000   
                             135.0    1.000 22000.000       NaN 22000.000   
                             228.0

**The hp of cars changes by theirs make_models and body_types most of the time. So I have decided to fill missing values with mode value of related group.**

In [95]:
df["hp"].replace(["-"], np.nan, inplace=True)

In [96]:
fill(df, "make_model", "body_type", "hp", "mode")

Number of NaN :  0
------------------
85.000     2543
66.000     2124
81.000     1403
100.000    1314
110.000    1113
70.000      890
125.000     711
51.000      696
55.000      589
118.000     550
92.000      466
121.000     392
147.000     380
77.000      353
56.000      294
54.000      276
103.000     253
87.000      232
165.000     194
88.000      177
60.000      160
162.000      98
74.000       81
96.000       72
71.000       59
101.000      47
67.000       40
154.000      39
122.000      35
119.000      30
164.000      27
135.000      24
52.000       22
82.000       22
1.000        20
78.000       20
294.000      18
146.000      18
141.000      16
57.000       10
120.000       8
104.000       8
112.000       7
191.000       7
155.000       6
117.000       6
184.000       5
65.000        4
90.000        4
76.000        4
168.000       3
98.000        3
149.000       3
80.000        3
93.000        3
167.000       2
228.000       2
53.000        2
143.000       2
150.000       2
14

### 21) Displacement_cc - Weight_kg

In [97]:
fill(df, "make_model", "body_type", "Displacement", "mode")

Number of NaN :  0
------------------
1598.000     4871
999.000      2446
1398.000     1326
1399.000      753
1229.000      678
1956.000      670
1461.000      638
1490.000      559
1422.000      467
1197.000      355
898.000       351
1395.000      320
1968.000      301
1149.000      288
1600.000      258
1618.000      212
1798.000      210
1498.000      196
1400.000      136
1248.000      110
1997.000      103
1364.000      102
1500.000       96
998.000        72
2000.000       68
1000.000       56
1200.000       51
1300.000       38
1998.000       25
1.000          22
2480.000       20
1984.000       18
1397.000       11
899.000        11
900.000         9
160.000         6
1499.000        5
929.000         5
1596.000        4
997.000         4
1199.000        3
1599.000        3
139.000         3
1396.000        3
1589.000        2
995.000         2
1495.000        2
1580.000        1
890.000         1
1995.000        1
1333.000        1
1100.000        1
54.000          1
1533.000

In [98]:
fill(df, "make_model", "body_type", "Weight", "mode")

Number of NaN :  0
------------------
1163.000    1582
1360.000    1419
1487.000     966
1135.000     837
1425.000     744
1180.000     694
1273.000     656
1165.000     603
1503.000     561
1734.000     556
1087.000     291
1335.000     242
1365.000     211
1199.000     205
1350.000     156
1119.000     153
1355.000     136
1260.000     127
1280.000     127
1275.000     112
1278.000     110
1255.000     108
1200.000     107
1522.000     103
1659.000     102
1195.000      96
1120.000      93
1403.000      91
1701.000      87
1250.000      85
1685.000      83
1441.000      82
1308.000      80
1285.000      80
1613.000      75
1110.000      75
1279.000      72
1364.000      70
1345.000      67
1733.000      65
1325.000      64
1141.000      64
1209.000      64
1071.000      64
1230.000      63
1845.000      56
1090.000      54
1052.000      53
1154.000      52
1664.000      52
1513.000      51
1065.000      50
1237.000      49
1440.000      46
1088.000      46
1205.000      46
1265.000  

### 22) CO2 Emission

In [99]:
df["CO2 Emission"].fillna("-", inplace=True)

In [100]:
df.groupby(["make_model", "body_type", "Fuel", "CO2 Emission"]).price.describe()

count      mean       std  \
make_model     body_type     Fuel    CO2 Emission                               
Audi A1        Compact       Benzine 97.0          77.000 15115.636  1746.383   
                                     98.0          23.000 17077.565  3010.233   
                                     100.0          3.000 15903.333  1859.901   
                                     102.0        106.000 17401.340  3100.205   
                                     103.0         30.000 21233.033  1794.551   
                                     104.0         92.000 18070.522  3280.132   
                                     105.0         12.000 22462.333   620.017   
                                     106.0         26.000 22064.192  1048.442   
                                     107.0          7.000 16771.429  2243.126   
                                     108.0         39.000 23943.795  3148.983   
                                     109.0          5.000 20951.800  3348.295   
                                     110.0         51.000 26340.373  2434.738   
                                     111.0        138.000 23765.283  3477.921   
                                     112.0         25.000 19960.360  2676.262   
                                     113.0          5.000 20388.000  2796.188   
                                     114.0          1.000 15490.000       NaN   
                                     115.0          7.000 24067.857  6950.650   
                                     116.0         20.000 23580.950  2813.103   
                                     117.0          8.000 19094.875  2942.954   
                                     118.0          2.000 13615.000    91.924   
                                     119.0          1.000 12550.000       NaN   
                                     120.0         27.000 15850.000     0.000   
                                     121.0          2.000 18710.000  3224.407   
                                     122.0          1.000 20881.000       NaN   
                                     123.0         17.000 19308.647  3816.070   
                                     126.0          1.000 19999.000       NaN   
                                     127.0          2.000 19888.000     0.000   
                                     134.0          2.000 22495.000   841.457   
                                     136.0          2.000 29088.000   128.693   
                                     137.0          2.000 28660.000   282.843   
                                     142.0          2.000 28750.000   268.701   
                                     331.0          1.000 12990.000       NaN   
                                     -             69.000 19491.623  4178.437   
                             Diesel  90.0           1.000 12880.000       NaN   
                                     91.0           5.000 13038.000  1108.927   
                                     92.0           1.000 12479.000       NaN   
                                     94.0           5.000 13960.000  1232.071   
                                     97.0          36.000 16163.889  3427.651   
                                     98.0          10.000 15417.000   663.610   
                                     99.0          19.000 16043.947  2257.403   
                                     100.0          1.000 16800.000       NaN   
                                     102.0         41.000 15771.902  1567.307   
                                     104.0         34.000 15627.941  1847.292   
                                     105.0          2.000 20870.000    42.426   
                                     106.0         17.000 16417.529  2647.273   
                                     107.0          6.000 16243.333   289.183   
                                     108.0          3.000 18950.000  3234.625   
                                     109.0         29.000 16353.034  1621.083   
  

In [101]:
df["CO2 Emission"].replace("-",np.NaN,inplace=True)

In [102]:
df['CO2 Emission'] = df['CO2 Emission'].fillna(df.groupby(['make_model','Displacement'])['CO2 Emission'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan))

In [103]:
df['CO2 Emission'] = df['CO2 Emission'].fillna(df.groupby(['body_type'])['CO2 Emission'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan))

In [104]:
df['CO2 Emission'] = df['CO2 Emission'].fillna(df.groupby(['body_type'])['CO2 Emission'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan))

In [105]:
df["CO2 Emission"].value_counts(dropna=False)

120.000    1116
97.000      836
99.000      753
114.000     594
119.000     581
102.000     532
104.000     501
85.000      464
107.000     458
103.000     445
124.000     426
128.000     414
106.000     377
117.000     362
108.000     362
127.000     302
126.000     288
110.000     286
150.000     282
113.000     272
118.000     270
140.000     245
111.000     237
139.000     236
109.000     234
129.000     215
141.000     209
105.000     205
135.000     204
92.000      198
112.000     186
130.000     180
123.000     178
143.000     171
145.000     167
134.000     165
95.000      161
116.000     157
136.000     152
98.000      151
133.000     145
153.000     139
137.000     133
125.000     132
149.000     117
147.000     113
101.000     105
132.000     100
115.000      91
122.000      83
121.000      82
138.000      75
93.000       66
168.000      58
142.000      58
100.000      56
187.000      53
148.000      48
131.000      48
144.000      40
154.000      40
146.000      37
94.000  

In [106]:
df.columns

Index(['make_model', 'body_type', 'price', 'km', 'hp', 'Type',
       'Previous Owners', 'Inspection new', 'Paint Type', 'Nr. of Doors',
       'Nr. of Seats', 'Gearing Type', 'Displacement', 'Weight', 'Drive chain',
       'Fuel', 'CO2 Emission', 'Comfort & Convenience',
       'Entertainment & Media', 'Extras', 'Safety & Security', 'Gears', 'age',
       'Upholstery_type', 'Consumption_combined', 'Consumption_city',
       'Consumption_country'],
      dtype='object')

### 23) Comfort_Convenience-Entertainment_Media-Extras-Safety_Security

In [107]:
fill(df, "make_model", "body_type", "Comfort & Convenience", "mode")

Number of NaN :  0
------------------
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                                   388
Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system                                                                                                                                       

In [108]:
fill(df, "make_model", "body_type", "Entertainment & Media", "mode")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1738
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                    1134
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                          1010
On-board computer                                                                                                  615
Radio                                                                                                              558
Bluetooth,Hands-free equipment,On-board computer,Radio                                                             515
On-board computer,Radio                                                                                            487
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               466
Bluetooth,

In [109]:
fill(df, "make_model", "body_type", "Extras", "mode")

Number of NaN :  0
------------------
Alloy wheels                                                                                                                                   5786
Alloy wheels,Touch screen                                                                                                                       697
Roof rack                                                                                                                                       596
Alloy wheels,Voice Control                                                                                                                      582
Alloy wheels,Touch screen,Voice Control                                                                                                         544
Alloy wheels,Roof rack                                                                                                                          529
Alloy wheels,Sport seats                                                  

In [110]:
fill(df, "make_model", "body_type", "Safety & Security", "mode")

Number of NaN :  0
------------------
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                      729
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                                 480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction contro

### 24) Consumption Combined

In [111]:
df['Consumption_combined'].value_counts(dropna=False)

NaN       2033
5.400      770
3.900      733
4.000      713
5.100      657
4.400      623
5.600      618
4.700      602
3.800      585
4.800      546
5.000      545
4.500      523
5.200      454
4.200      435
4.600      426
4.900      393
5.500      380
5.300      380
3.700      369
5.900      369
4.100      342
5.700      342
6.000      331
4.300      307
3.300      307
3.500      288
6.200      216
3.600      194
6.300      181
6.100      175
5.800      164
6.600      148
6.800      136
3.400      106
6.400       75
3.000       69
7.400       66
6.500       43
6.700       43
7.100       38
10.000      34
6.900       27
3.200       25
8.300       20
7.600       14
7.000       10
3.100        7
7.200        6
7.800        6
8.000        5
51.000       4
8.600        4
7.900        3
8.700        3
1.600        3
7.300        2
8.100        2
40.000       2
38.000       2
0.000        2
9.100        1
43.000       1
7.500        1
13.000       1
55.000       1
54.000       1
1.200     

In [112]:
coms = ((df['Consumption_city']+df['Consumption_country'])/2)

In [113]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(coms)

In [114]:
mode_Cons= df.groupby(['CO2 Emission'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [115]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons)

In [116]:
mode_Cons1= df.groupby(['Displacement'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)                


In [117]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons1)

In [118]:
mode_Cons2= df.groupby(['make_model'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [119]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons2)

In [120]:
df['Consumption_combined'].isnull().sum()

0

In [121]:
df.drop(["Consumption_country","Consumption_city"],axis=1,inplace=True)

In [122]:
df.drop("hp",axis=1,inplace=True)

In [123]:
df.to_csv("missings_filled.csv", index=False)

In [124]:
df.isnull().sum()

make_model               0
body_type                0
price                    0
km                       0
Type                     0
Previous Owners          0
Inspection new           0
Paint Type               0
Nr. of Doors             0
Nr. of Seats             0
Gearing Type             0
Displacement             0
Weight                   0
Drive chain              0
Fuel                     0
CO2 Emission             0
Comfort & Convenience    0
Entertainment & Media    0
Extras                   0
Safety & Security        0
Gears                    0
age                      0
Upholstery_type          0
Consumption_combined     0
dtype: int64